Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as np
import os
from statsmodels.tsa.seasonal import seasonal_decompose

Importando os dados

In [2]:
path_hist = 'C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\hist_cotacoes\\'
lista_historicos = os.listdir(path_hist)

Removendo historicos inexistentes e/ou inconsistentes

In [3]:
lista_tickers_removidos = ["APTI4", "BPAR3", "MMAQ4", "MMAQ3"]

lista_tickers_removidos = lista_tickers_removidos

lista_hist_removidos = [f"hist_{x}.csv" for x in lista_tickers_removidos]

for historico in lista_historicos:

    # print(path_hist + historico)
    if historico in lista_hist_removidos:
        os.remove(path_hist + historico)

lista_historicos = os.listdir(path_hist)

Consolidando e tratantando historicos de cotações

In [4]:
lista_tickers = [f"{historico}".replace(".csv", "").replace("hist_", "") for historico in lista_historicos]
# removendo histórico do ibov
lista_tickers.remove("ibov")

# Criando um dataframe com todas as datas como um índice
data_inicial = '2013-01-01'
data_final = '2023-10-01'
datas = pd.date_range(start=data_inicial, end=data_final, freq='D')
df_hist_cot = pd.DataFrame({'Date': datas}).set_index('Date')

for ticker in lista_tickers:

    ticker_invalido = False

    # Importantdo histórico do ticker
    df_hist_cot_ticker = pd.read_csv(f"C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\data\\hist_cotacoes\\hist_{ticker}.csv")
    df_hist_cot_ticker["Date"] = pd.to_datetime(df_hist_cot_ticker["Date"])
    df_hist_cot_ticker = df_hist_cot_ticker.set_index("Date")
    
    # Validadno consistência dos dados de cotação
    for cotacao in df_hist_cot_ticker.index:

        if df_hist_cot_ticker.loc[cotacao]["Adj Close"] < 0:
            ticker_invalido = True
    
    if ticker_invalido == True:
        os.remove(path_hist + f"hist_{ticker}.csv")
        continue

    # Inserindo histórico do ticker no dataframe de históricos de cotação
    for data in df_hist_cot_ticker.index:
        df_hist_cot.loc[data, ticker] = df_hist_cot_ticker.loc[data, "Adj Close"]

# Removendo datas sem cotação (finais de semana, feriados, etc.)
df_hist_cot.dropna(inplace=True, how="all")

# Tratando datas pré ipo
for ticker in df_hist_cot.columns:

    data_ipo = df_hist_cot[ticker].first_valid_index()
    for data in df_hist_cot.index:

        if data < data_ipo:
            
            df_hist_cot.loc[data, ticker] = 0

# Tratando cotações faltantes
for ticker in df_hist_cot.columns:
    
    df_hist_cot[f"{ticker}"].ffill(inplace=True)

# df_hist_cot

C:\Users\victo\AppData\Local\Temp\ipykernel_25364\1836375325.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_hist_cot.loc[data, ticker] = df_hist_cot_ticker.loc[data, "Adj Close"]
C:\Users\victo\AppData\Local\Temp\ipykernel_25364\1836375325.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_hist_cot.loc[data, ticker] = df_hist_cot_ticker.loc[data, "Adj Close"]
C:\Users\victo\AppData\Local\Temp\ipykernel_25364\1836375325.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling

Exportando historicos de cotações

In [5]:
df_hist_cot.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_hist_cot.csv")

Decompondo as séries temporais dos históricos de cotação

In [6]:
df_tend = pd.DataFrame(index = df_hist_cot.index)
df_saz = pd.DataFrame(index = df_hist_cot.index)
df_resid = pd.DataFrame(index = df_hist_cot.index)

for ticker in df_hist_cot.columns:

    # 22 dias de cotação por mês / 252 dias de cotação por ano / df_media_segm.shape[0]//2 maior período possível para decomposição da base de dados
    
    df_tend[f"{ticker}"] = seasonal_decompose(df_hist_cot[f"{ticker}"],
                                                period = 252,
                                                model = "additive").trend
    df_saz[f"{ticker}"] = seasonal_decompose(df_hist_cot[f"{ticker}"],
                                                period = 252,
                                                model = "additive").seasonal
    df_resid[f"{ticker}"] = seasonal_decompose(df_hist_cot[f"{ticker}"],
                                                period = 252,
                                                model = "additive").resid

df_tend.dropna(inplace=True)
df_saz.dropna(inplace=True)
df_resid.dropna(inplace=True)

C:\Users\victo\AppData\Local\Temp\ipykernel_25364\1463062443.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_tend[f"{ticker}"] = seasonal_decompose(df_hist_cot[f"{ticker}"],
C:\Users\victo\AppData\Local\Temp\ipykernel_25364\1463062443.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_saz[f"{ticker}"] = seasonal_decompose(df_hist_cot[f"{ticker}"],
C:\Users\victo\AppData\Local\Temp\ipykernel_25364\1463062443.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

Exportando as componentes das séries temporais do histórico de cotações

In [7]:
df_tend.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_tend.csv")
df_saz.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_saz.csv")
df_resid.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_resid.csv")

Gerando histórico de cotações normalizado

In [8]:
df_hist_cot_norm = pd.DataFrame(index = df_hist_cot.index)
for ticker in df_hist_cot.columns:

    data_ref = df_hist_cot[f"{ticker}"][df_hist_cot[f"{ticker}"] != 0].sort_index(ascending=True).index[0]
    
    df_hist_cot_norm[f"{ticker}"] = df_hist_cot[f"{ticker}"] / df_hist_cot.loc[data_ref][f"{ticker}"]

# df_hist_cot_norm

C:\Users\victo\AppData\Local\Temp\ipykernel_25364\393239579.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_hist_cot_norm[f"{ticker}"] = df_hist_cot[f"{ticker}"] / df_hist_cot.loc[data_ref][f"{ticker}"]
C:\Users\victo\AppData\Local\Temp\ipykernel_25364\393239579.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_hist_cot_norm[f"{ticker}"] = df_hist_cot[f"{ticker}"] / df_hist_cot.loc[data_ref][f"{ticker}"]
C:\Users\victo\AppData\Local\Temp\ipykernel_25364\393239579.py:6: PerformanceWarning: DataFrame is highly fragment

In [9]:
df_hist_cot_norm.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_hist_cot_norm.csv")

Decompondo as séries temporais dos hitoricos de cotação normalizados

In [10]:
df_tend_norm = pd.DataFrame(index = df_hist_cot_norm.index)
df_saz_norm = pd.DataFrame(index = df_hist_cot_norm.index)
df_resid_norm = pd.DataFrame(index = df_hist_cot_norm.index)

for ticker in df_hist_cot.columns:

    # 22 dias de cotação por mês / 252 dias de cotação por ano / df_media_segm.shape[0]//2 maior período possível para decomposição da base de dados
    
    df_tend_norm[f"{ticker}"] = seasonal_decompose(df_hist_cot_norm[f"{ticker}"],
                                                period = 252,
                                                model = "additive").trend
    df_saz_norm[f"{ticker}"] = seasonal_decompose(df_hist_cot_norm[f"{ticker}"],
                                                period = 252,
                                                model = "additive").seasonal
    df_resid_norm[f"{ticker}"] = seasonal_decompose(df_hist_cot_norm[f"{ticker}"],
                                                period = 252,
                                                model = "additive").resid

df_tend_norm.dropna(inplace=True)
df_saz_norm.dropna(inplace=True)
df_resid_norm.dropna(inplace=True)

C:\Users\victo\AppData\Local\Temp\ipykernel_25364\2724226793.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_tend_norm[f"{ticker}"] = seasonal_decompose(df_hist_cot_norm[f"{ticker}"],
C:\Users\victo\AppData\Local\Temp\ipykernel_25364\2724226793.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_saz_norm[f"{ticker}"] = seasonal_decompose(df_hist_cot_norm[f"{ticker}"],
C:\Users\victo\AppData\Local\Temp\ipykernel_25364\2724226793.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of c

Exportdo as componentes das séries temporais do hitórico de cotações normalizados

In [11]:
df_tend_norm.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_tend_norm.csv")
df_saz_norm.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_saz_norm.csv")
df_resid_norm.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_resid_norm.csv")

Gerando a matriz de correlação entre as empresas

In [18]:
mtx_corr_tickers_norm = df_hist_cot_norm.corr()
mtx_corr_tickers_norm.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\mtx_corr_tickers_norm.csv")

Gerando a matriz de correlação entre as sazonalidades das empresas

In [19]:
mtx_corr_saz_tickers_norm = df_saz_norm.corr()
mtx_corr_saz_tickers_norm.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\mtx_corr_saz_tickers_norm.csv")